# A Quick Note on GPU Accuracy and Double Precision

Often when optimizing simulation software, we run into a classic roadblock:  *Floating point precision is not accurate enough, we must use double precision.*

Of course, there are many instances in real physical problems where indeed, a solution requires double precision computation.  However, we should be careful not to fall into a trap - the performance benefits of using 32bit vs 64bit datatypes is **significant**

Particularly on gpus, where memory and bandwidth charge a higher premium, 32bit operations are much more desireable (see also Machine Learning's recent push to use 16 bit operations).

But we do have one fascinating edge on our side when using GPU calculations: they are often more accurate than CPU calculations.

No, I don't mean to say that the GPU somehow has a more accurate FPU.  Instead, due to the massively number of GPU threads and correspondingly small units of work, GPU threads more often deal with numbers of comparable sizes and are thus more accurate.

## Example: Sum of a double precision array.

Take as a simple example reducing the sum of an array.  A classic CPU loop will look something like:

    sum = 0
    for element in array:
      sum += element
    return sum

As the sum gets larger, so also does the error.

For a GPU, we will often use a tree based approach:
<img src="tree.png" width="300">

Due to the tree like nature, the elements sent to the FPU tend to have equivalent sizes and the sum operation therefore has less error.

Lets look at a simple examply using *numpy* and *pycuda*   


In [ ]:
import numpy as np
import pycuda.autoinit
import pycuda.driver as cuda
import pycuda.gpuarray as gpuarray

In [ ]:
N = 1000000

#source double precision array
d = np.random.rand(N) 

#floating point CPU copy
f = d.astype(np.float32)

#floating point GPU copy
g = gpuarray.to_gpu(f)

#sums
dsum = d.sum()
fsum = f.sum()
gsum = gpuarray.sum(g).get()

#distance from double
fd = np.abs(dsum-fsum)
gd = np.abs(dsum-gsum)

In [ ]:
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings("ignore")
plt.plot([0, 0],[0, fd], c='blue', label='cpu distance: %f' %(fd))
plt.plot([0, gd],[0, 0], c='green', label='gpu distance: %f' %(gd))
ax = plt.axes()
ax.set_xbound(-0.2 * fd, 1.1 * fd)
ax.set_ybound(-0.2 * fd, 1.1 * fd)
plt.legend()

print("True sum:", dsum)
print("CPU 32bit precision:", fsum)
print("GPU 32bit precision:", gsum)
plt.show()

## Conclusion

And there you have it.  The GPU 32bit sum is quite a bit closer to the true double precision sum than CPU 32bit sum.

Here we have a simple example with a simple reduction, but in reality there are many more examples where we can relax our double precision requirement.

In [ ]:
print(dsum,fsum,gsum)
print(fsum-49947.5, gsum-49947.5)